In [58]:
from newspaper import Article
url = 'http://www.zhihu.com/question/47414054/answer/105880816'
url = 'https://music.douban.com/topic/postrock?qq-pf-to=pcqq.discussion'
# url = 'http://news.sina.com.cn/zl/2016-06-16/doc-ifxtfrrf0436412.shtml'
a = Article(url, language='zh') # Chinese
a.download()
a.parse()

Building prefix dict from C:\Anaconda3\lib\site-packages\jieba\dict.txt ...
--- Logging error ---
Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 982, in emit
    self.flush()
  File "C:\Anaconda3\lib\logging\__init__.py", line 962, in flush
    self.stream.flush()
OSError: [Errno 22] Invalid argument
Call stack:
  File "C:\Anaconda3\lib\runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 596, in launch_instance
    app.start()
  File "C:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "C:\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 162, in start
    super(ZMQIOLoop, self).sta

In [60]:
print(a.title)
print(a.text)
# print(a.html)
print(a.authors)
print(a.top_image)
print(a.movies)

后摇恩仇录 The Odyssey of Post-Rock
关键词：颠覆、意外、实验、拿来主义

欢迎来到1991年，后摇纪元元年，充满叛逆精神的“后摇1.0”年代从此开启！标志性的事件是两张专辑的发行：Talk Talk的《Laughing Stock》和Slint的《Spiderland》。这两张专辑无论是在音乐背景还是在风格上来讲都非常不同，但各自破旧立新的胆识和魄力都对后摇都产生了极其深远的影响 （有趣的是，这两张开创了后摇纪元的经典专辑也都是两支乐队的最后一张专辑。莫非这是先驱者的象征意味么？）

《Spiderland》和《Laughing Stock》

Allmusic评价Slint的《Spiderland》：“这张专辑简直是这几位乐手应该被抓进疯人院的证据。”[5] 它让人无所适从：没有传统摇滚乐里让人难忘的吉他riff或者solo, 没有提神醒脑的鼓点，一个阴沉的声音低低私语着... 《Spiderland》里的歌曲颠覆着所有人的听觉习惯：你不再能够听得出一个个性鲜明的吉他手或者是鼓手，音乐是氛围化情绪化的，所有的配器（包括人声在内）都仅仅是为了达到这一目的——就像当你看到一幅画很难看到单独的笔触。在Slint的第一张专辑《Tweez》中尚且能明显分辨出后硬核和数学摇滚的影响，但在第二张专辑里则出现了质的飞跃：乐队摆脱了摇滚乐的重力束缚，飞向了另一个境界，尽管这个境界是一个黑暗阴森的梦境。

如果说Slint的进步是一群抑郁书呆终于达到了病态美的极致的话，Talk Talk走的可是截然相反的路线：从世俗光鲜合成器大流行蓦然回首证悟终于获得神性。八十年代就开始在欧洲大红的Talk Talk是浮夸浪漫的新浪潮一员，首专《Party's Over （1982）》里有着轰鸣的鼓机鼓点，自带迪厅光线般七彩霓虹的合成器，Gary Numan式顿挫的唱腔。但在不到十年的时间里，乐队已经变成了另一个样子：《Laughing Stock》有着复杂配器，精细微妙的制作，包括主唱此时已经几近虔诚的演唱都是如此有机地融合在一起，散发着美与爱、包容的微光。用“圣洁”这个词来形容一张专辑，不管是摇滚还是后摇滚似乎都不太合适，但《Laughing Stock》营造出来的氛围就具有这样的冲击力。（据说主唱Mark Hollis坚持在录音时点蜡烛熏香。[6] 但这也不算什么，几

In [34]:
def h2t_handle_html(html):
  import html2text
  h2t = html2text.HTML2Text()
  h2t.body_width = 0
  return '\n'.join(p.rstrip() for p in h2t.handle(html).strip().split('\n'))


In [2]:
from zhihu_oauth import ZhihuClient
TOKEN_FILE = 'token.pkl'
client = ZhihuClient()
client.load_token(TOKEN_FILE)

me = client.me()

print('name', me.name)
print('headline', me.headline)
print('description', me.description)

answer = client.answer(94150403)
answer


name Pobpob
headline 试试知乎有什么厉害的
description 


In [5]:
answer._build_url()

'https://api.zhihu.com/answers/94150403'

In [10]:
r = client.test_api('GET', 'https://api.zhihu.com/answers/94150403')
s = str(r.content, encoding='utf-8')
from pprint import pprint
from pylon import puts
pprint(s)

('{"question":{"author":{"name":"匿名用户","url":"","user_type":"people","headline":"","avatar_url":"http:\\/\\/pic2.zhimg.com\\/aadd7b895_s.jpg","type":"people","id":"0"},"url":"https:\\/\\/api.zhihu.com\\/questions\\/42248369","title":"如何评价南开大学津南校区的建设质量？","answer_count":68,"type":"question","id":42248369},"is_copyable":true,"can_comment":{"status":true,"reason":""},"is_mine":false,"author":{"badge":[],"name":"7sDream","url":"https:\\/\\/api.zhihu.com\\/people\\/ef7be83f9f1fcab3adfa7a6a1f510427","gender":1,"user_type":"people","headline":"二次元普通居民，不入流程序员，http:\\/\\/0v0.link","avatar_url":"http:\\/\\/pic3.zhimg.com\\/50\\/893fd554c8aa57196d5ab98530ef479a_s.jpg","type":"people","id":"ef7be83f9f1fcab3adfa7a6a1f510427"},"url":"https:\\/\\/api.zhihu.com\\/answers\\/94150403","comment_permission":"all","excerpt":"卸了个腰…… '
 '@gygigi khgjkbj '
 '我是个挺容易满意的人，总体来说我觉得其实还行。优点（或者在慢慢进步的地方）：寝室比老校区的好！新图书馆好评，借书机遍地都是，更方便了。自习地方更多，也更舒服。还有个视听区还是多媒体区，可以\\"放松自己\\"。文理之间草坪里的小路，终于听劝的从扯蛋石板路改成了按摩石子路。以前理科组团没超市，现在有了 '
 

In [41]:
client._session
#  |  __attrs__ = ['headers', 'cookies', 'auth', 'proxies', 'hooks', 'params...
#  |  
#  |  ----------------------------------------------------------------------
#  |  Methods inherited from SessionRedirectMixin:
#  |  
client._session.auth

In [29]:
answer = client.answer(106335325)
comments = listb(answer.comments)
for i, c in enumerate(comments, 1):
    print(i, c.content)

1 地主家也没有余粮啊
2 所以到最后便成了赚人民币花美元的锅咯
3 这实际上就是吸血效应
4 →_→一百多年前的事情重来
5 这个论点我记得马克思说过的，更高房价催生的是更庞大的产业后备军
6 让一部分人先买的起房。
7 玄处的文风……玄体
8 相比起马督工的洗地文，还是玄处的回答最好！
9 药丸
10 看不到武装革命可能性的情况下,读马列是不是会越读越郁闷?
11 无产阶级联合起来！
12 嗯，你个缓则，居然最后骂哮喘，人家容易吗？
13 可以用马列主义指导炒房嘛，郁闷啥呢？
14 早就说过，很多人以为2块钱的地铁是福利，那他妈的是为了让你住的更远。
15 活着真难
16 自从关注了玄处，督工的文章我已经懒得看了。
17 地铁还好啦，这点涨价翻不起多少浪花。可怕的是廉租房啊，这是用你交的税去盖房子，以便从外地再弄来一批屌丝和你抢饭碗。
18 还有火车票20年不涨价，很多人大呼良心，明显脑子缺根弦。
19 你觉得能问出这话的会是有钱炒房的阶级么...
20 意思是首都全国吸血，美国全世界吸血?
21 这个论点很有意思，感谢提供了一个新视角
22 如果天朝最终能够兵不血刃的征服（买下）整个美利坚，赵老爷家真是功在社稷，利在千秋啊
23 咋滴，准备闹革命？图样图森破。<br>无产阶级只有解放了全人类才能解放自己，各位以及各位的孙子都没希望。<br>丛林太危险，到草原就安全了。
24 地铁修到燕郊，意味着晚上燕郊族们只有多加班一个小时，才能把因为新地铁通车而新搬到三河、大厂来的外来务工人员PK下去。这句话有点疑问，问题是能不能PK掉别人是看你的水平，不是看你的加班时间，也不是看你的“工作时间+通勤时间”啊
25 玄处你个吃皇粮的处级干部说出这等话来，我替赵老爷和钱老爷们伤心呐，白养了
26 当我们谈论竞争对手的时候，指的是和同等水平的人竞争。意思是说，三河大厂忽然搬来一群和你水平相当的人，看你怎么破。
27 就拿手机来说，美国人负责操作系统以及关键CPU的设计生产掌握最核心的技术，日韩台负责次等配件的零件生产，中国负责组装，亚非拉提供原材料和市场。世界和谐~~
28 玄处什么都懂呀
29 问题在于，竞争不是“你们俩工作一个月，创造效益少的人滚蛋”（简称A），而是“你俩一起过来面试，水平高的留下，如果你创造的价值匹配了我给你发的钱，你就一直干下去”（简称B），如果竞争方式

In [39]:
comments
top_voteup_comment = max(comments, key=lambda x: x.vote_count)
top_voteup_comment
top_voteup_comment.content

'早就说过，很多人以为2块钱的地铁是福利，那他妈的是为了让你住的更远。'

In [45]:
url = 'https://api.zhihu.com/comments/79231231/'
client._session.request('GET', url)

RetryError: HTTPSConnectionPool(host='api.zhihu.com', port=443): Max retries exceeded with url: /comments/79231231/ (Caused by ResponseError('too many 404 error responses',))

In [54]:
comment_list_id = '14852408'
url = 'http://www.zhihu.com/node/AnswerCommentBoxV2?params=%7B%22answer_id%22%3A%22{}%22%2C%22load_all%22%3Atrue%7D'.format(comment_list_id)
r = client._session.get(url)
rc = r.content

In [56]:
str(rc, encoding = "utf-8")

'<div class="zm-comment-box" data-count="21">\n<i class="icon icon-spike zm-comment-bubble"></i>\n<a class="zg-anchor-hidden" name="comment-0"></a>\n<div class="zm-comment-list">\n<div class="zm-item-comment" data-id="79700253" tabindex="-1" aria-label="评论">\n<a class="zg-anchor-hidden" name="comment-79700253"></a>\n<a title="庄鸡仔"\ndata-tip="p$t$zhuang-ji-zi-22"\nclass="zm-item-link-avatar"\nhref="/people/zhuang-ji-zi-22">\n<img src="https://pic2.zhimg.com/d079f353650d9c48f4f60a51f6a87f75_s.jpg" class="zm-item-img-avatar">\n</a>\n<div class="zm-comment-content-wrap">\n<div class="zm-comment-hd">\n<a data-tip="p$t$zhuang-ji-zi-22" href="http://www.zhihu.com/people/zhuang-ji-zi-22" class="zg-link" title="庄鸡仔"\n>庄鸡仔</a>\n\n</div>\n<div class="zm-comment-content">\n233政治智慧。。。。\n</div>\n<div class="zm-comment-ft">\n<span class="date">2015-06-01</span>\n\n\n<a href="#" class="reply zm-comment-op-link" name="reply_comment">\n<i class="zg-icon zg-icon-comment-reply"></i>回复</a>\n\n\n<a href="#"

In [32]:
c0 = comments[0]
c12 = comments[12]
print(c12.content)
g= c0.conversation
print(g)
replies = list(c0.replies)
[print(r.content) for r in replies]
print()


replies12 = list(c12.replies) # 所有回复本评论的评论, 第一条为本评论
[print(r.author.name, r.content) for r in replies12]
print()


g12 = list(c12.conversation)  # 包含该评论的对话, 从最开始到结束
[print(r.author.name, r.content) for r in g12]

可以用马列主义指导炒房嘛，郁闷啥呢？
地主家也没有余粮啊

玄不救非氪不改命 可以用马列主义指导炒房嘛，郁闷啥呢？
Razor Liu 你觉得能问出这话的会是有钱炒房的阶级么...
暗黑的傀儡师 思路很新颖，就是把劳动力市场看的太简单了，不是还有简单劳动和复杂劳动之分，住燕郊的人大部分也不是富士康那种厂工，所以工作时间和产出之间并不是线性的；另外，老板给租了房，员工很可能更不愿意加班了。

Razor Liu 看不到武装革命可能性的情况下,读马列是不是会越读越郁闷?
玄不救非氪不改命 可以用马列主义指导炒房嘛，郁闷啥呢？
Razor Liu 你觉得能问出这话的会是有钱炒房的阶级么...


[None, None, None]

In [14]:
from newspaper import Article
url = 'http://mp.weixin.qq.com/s?__biz=MzIyNDA2NTI4Mg==&mid=2655408046&idx=3&sn=1e6017387b5eea9dd51c3bd04f057c79&scene=0#rd'
# 时空的观念：艺术学与物理学

url = 'http://mp.weixin.qq.com/s?__biz=MzA3OTgzMzUzOA==&mid=403194184&idx=1&sn=2b6ff0c0b844baaf3ed55cd0c684af0c&scene=0#rd'
# Google的深度学习强在哪？谷歌首席科学家说了这些奇妙特性
article = Article(url, language='zh')

article.download()
article.parse()

# print(article.text[:1000], '......')
# print(article.title)
# print(article.top_image)
# print(article.authors)

In [17]:
help(article)

Help on Article in module newspaper.article object:

class Article(builtins.object)
 |  Article objects abstract an online news article page
 |  
 |  Methods defined here:
 |  
 |  __init__(self, url, title='', source_url='', config=None, **kwargs)
 |      The **kwargs argument may be filled with config values, which
 |      is added into the config object
 |  
 |  build(self)
 |      Build a lone article from a URL independent of the source (newspaper).
 |      Don't normally call this method b/c it's good to multithread articles
 |      on a source (newspaper) level.
 |  
 |  build_resource_path(self)
 |      Must be called after computing HTML/final URL
 |  
 |  download(self, html=None)
 |      Downloads the link's HTML content, don't use if you are batch async
 |      downloading articles
 |  
 |  fetch_images(self)
 |  
 |  get_parse_candidate(self)
 |      A parse candidate is a wrapper object holding a link hash of this
 |      article and a final_url of the article
 |  
 |  ge

In [26]:
article.article_html

''

In [13]:


import re
import json
import sys
import random
import time
from pyquery import PyQuery

import requests
url = 'http://mp.weixin.qq.com/s?__biz=MjM5NzE2NTY0Ng==&mid=403189023&idx=1&sn=8f5774a37f965d3e33901172d191ac91&scene=0#rd'
url = 'http://mp.weixin.qq.com/s?__biz=MzI3MDE0NzI0MA==&mid=2650991368&idx=2&sn=efc68055efff53061cedb8eba58bf5fd&scene=0#rd'
r = requests.get(url)
# print(r.status_code)
# print(r.headers['content-type'])
# print(r.encoding)

d = PyQuery(r.text)


In [15]:
t = d('div#js_article')
t.text()


def html_to_markdown(html):
  import html2text
  h2t = html2text.HTML2Text()
  h2t.body_width = 0
  r = h2t.handle(html).strip()
  r = '\n'.join(p.rstrip() for p in r.split('\n'))
  return re.sub('\n{4,}', '\n\n\n', r)

html_to_markdown(t.text())

'脱欧前后与百年以前的伦敦城｜政好读书 · 政见CNPolitics 2016-07-03 于灵歌 政见CNPolitics 政见CNPolitics 政见CNPolitics 微信号 cnpolitics2011 功能介绍 我们想拆掉知识的高墙，让普通人读懂社科学术研究，接触靠谱思想资源。 (function(){ var cover = "http://mmbiz.qpic.cn/mmbiz/gCibFJDQkkDyKdNgXciaVaAR2Srh09eBibaPtBfLMgTVqRm7VFZiaGeGZd4HyBgDBeRv6Tbc2Ky9zhZeyDCmaMOACg/0?wx_fmt=jpeg"; document.write(\'\'); })(); The Houses of Parliament, Sunset (Claude Monet, 1903) 主播： 于灵歌 / 政见团队成员 “如果你厌倦了伦敦，你就厌倦了生活。” 伦敦是一座极具生活意识的城市，也是一座属于世界的城市。尽管这一个月以来，脱欧前后的伦敦始终处于漩涡之中。 百年前，曾有一人亲自记下此处历史的行迹：梁启超。一战结束巴黎和会前后，梁启超曾与同僚一同有游历英国和欧陆，写下了《欧游心影录》，完成了从乡人、国人到世界人自我定位的转变。他在伦敦停留最久，笔下的 “巴力门（parliament）” 政治场景分外有趣，有对中英两国国民性的洞察，也有对当时民国政局的深刻反思；而他面对战后凋败的欧洲，尽管对科学万能与物质崇拜有所质疑，却从未对欧洲文明失去敬仰和信心。 在这一期政好读书节目中，来聊聊我心中最喜爱也是最伟大的城市之一，伦敦。当然，也看看百年前梁启超笔下的它，与欧洲、世界的模样。 “这个世界很像伦敦。它不好也不坏，只是就这个样子。这里没有道德和不光荣，只有你自己的孤单法则。直到这场竞赛开跑，直到它的结尾，直到我们成为这里的鬼魂。这就是我们的曾经。” 收听本期节目，请在 网易云音乐、考拉FM、喜马拉雅、凤凰FM 这几个播客平台中搜索 “政见” 或 “政好读书” 即可。我们在苹果 iTunes 也已经上线。 点击下方 “ 阅读原文 ” 可直接跳转网易云音乐网页版收听。 打赏支持政见运作 微信编辑：张烨 图片编辑：林亦然 最新合辑下载 百度网盘: http://t.cn/RGCasri GitH

In [27]:
url = 'http://mp.weixin.qq.com/s?__biz=MjM5NzE2NTY0Ng==&mid=403189023&idx=1&sn=8f5774a37f965d3e33901172d191ac91&scene=0#rd'
url = 'http://mp.weixin.qq.com/s?__biz=MzI3MDE0NzI0MA==&mid=2650991368&idx=2&sn=efc68055efff53061cedb8eba58bf5fd&scene=0#rd'

doc = PyQuery(url=url)('div#js_article')

doc.html()

'\r\n        \r\n        <div id="js_top_ad_area" class="top_banner">&#13;\n &#13;\n        </div>&#13;\n                &#13;\n&#13;\n        <div class="rich_media_inner">&#13;\n                        <div id="page-content">&#13;\n                <div id="img-content" class="rich_media_area_primary">&#13;\n                    <h2 class="rich_media_title" id="activity-name">&#13;\n                        脱欧前后与百年以前的伦敦城｜政好读书 · 政见CNPolitics &#13;\n                    </h2>&#13;\n                    <div class="rich_media_meta_list">&#13;\n                        \t\t\t\t\t\t                        <em id="post-date" class="rich_media_meta rich_media_meta_text">2016-07-03</em>&#13;\n&#13;\n                                                <em class="rich_media_meta rich_media_meta_text">于灵歌</em>&#13;\n                                                <a class="rich_media_meta rich_media_meta_link rich_media_meta_nickname" href="javascript:void(0);" id="post-user">政见CNPolitics</a>&#13;\n     

In [3]:
answer = client.answer(106335325)
answer

In [27]:
a = answer
# print(help(a))
a._get_data()


In [29]:
a._data

{'author': {'avatar_url': 'http://pic1.zhimg.com/50/196c382dad309ae5361a42cdb8d9b5e4_s.png',
  'badge': [],
  'gender': 1,
  'headline': '东欧政治与杨幂及王晓晨研究',
  'id': '15bf56e5ba4e3cb34980b67fc991f517',
  'name': '玄不救非氪不改命',
  'type': 'people',
  'url': 'https://api.zhihu.com/people/15bf56e5ba4e3cb34980b67fc991f517',
  'user_type': 'people'},
 'can_comment': {'reason': '', 'status': True},
 'comment_count': 630,
 'comment_permission': 'all',
 'content': '一线城市而论，城市中心区住宅的入住率相当高，怎么能说“人们买不起房子”？其实真相是你们买不起房子，而不是大家都买不起房子。具体而言，就是依靠工资收入的普通劳动者买不起房子。就连平均月薪2万的知乎侠，完全依靠工资收入，也是买不起市中心的房子的。<br><br>靠工资收入买不起房子，就对了。要知道，一线城市中心的房产，一部分是属于原住民和原来机关企事业单位的福利分房，这部分人现在已经是市中心房产与土地的供给者了。而实际上的新增购买者，主要就是外来购房者。其中一部分是外来投资购房，而更多的是外来工作者在家族资助下购房。但结果都是一样的，那就是一线城市的房产成为大城市从中小城市汲取资源的抽水机。<br><br>比如说，中小城市的老两口本来应该在当地消费的，结果节衣缩食用于给子女在一线城市购房，于是当地的消费就萎缩了；中小城市的资本家本来应该在当地扩大投资的，结果跑去一线城市购房，于是当地的投资就萎缩了。这是显性的。还有隐性的啊，比如说许多人经常抱怨华北、东北、西北的投资环境差，但是你知道有些贪官污吏为了在一线城市购房，刮起地皮来也是很拼的。贪污腐败一辈子，最后全部贡献给一线城市的楼市了。<br><br><a href="http://link.zhihu.com/?target=ht

In [85]:

def get_json_comments(answer, limit=20, offset=0):
  import json
  answer._build_url()
  url = answer._build_url() + '/comments'
  response = client.test_api('GET', url, params={'limit': limit, 'offset': offset})
  j = json.loads(str(response.content, encoding='utf-8'))
  return j['data'], j['paging']

In [91]:
import pprint
data, paging = get_json_comments(answer, limit=12, offset=12)

In [92]:
print(len(data))
paging

20


{'is_end': False,
 'next': 'https://api.zhihu.com/answers/106335325/comments?limit=20&offset=32',
 'previous': 'https://api.zhihu.com/answers/106335325/comments?limit=20&offset=0'}

In [142]:
client

In [144]:
import json
url = 'https://api.zhihu.com/answers/106335325/'
r = client.test_api('GET', url, params=None)
j = json.loads(str(r.content, encoding='utf-8'))
j

RetryError: HTTPSConnectionPool(host='api.zhihu.com', port=443): Max retries exceeded with url: /answers/106335325/ (Caused by ResponseError('too many 404 error responses',))

In [169]:
answer = client.answer(106335325)
answer

In [170]:
comments = list(answer.comments)

In [167]:
a = _
a

In [166]:
c = a

cv = list(c.conversation)
cv

RetryError: HTTPSConnectionPool(host='api.zhihu.com', port=443): Max retries exceeded with url: /comments/143135536/conversation (Caused by ResponseError('too many 400 error responses',))

In [136]:

def valuable_conversations(answer, group_limit=10):
  '''
  return [[comment, comment...], ...]

  replies12 = list(c12.replies) # 所有回复本评论的评论, 第一条为本评论
  [print(r.author.name, r.content) for r in replies12]
  # 玄不救非氪不改命 可以用马列主义指导炒房嘛，郁闷啥呢？
  # Razor Liu 你觉得能问出这话的会是有钱炒房的阶级么...
  # 暗黑的傀儡师 思路很新颖，就是把劳动力市场看的太简...

  g12 = list(c12.conversation)  # 包含该评论的对话, 从最开始到结束
  [print(r.author.name, r.content) for r in g12]
  # Razor Liu 看不到武装革命可能性的情况下,读马列是不是会越读越郁闷?
  # 玄不救非氪不改命 可以用马列主义指导炒房嘛，郁闷啥呢？
  # Razor Liu 你觉得能问出这话的会是有钱炒房的阶级么...
  '''
  def _parse_comment(comment):
    reply_to_author = ' 回复 **{}**'.format(comment.reply_to.name) if comment.reply_to else ''
    vote_count = '  ({} 赞)'.format(comment.vote_count) if comment.vote_count else ''
    content = (comment.content).strip()
    if '\n' in content:
      content = '\n\n' + content
    text = '**{}**{}: {}{}\n\n'.format(comment.author.name, reply_to_author, content, vote_count)
    return text


  all_comments = list(answer.comments)
  selected_ids = set()

  while group_limit:
    selecting = (c for c in all_comments if c.id not in selected_ids)
    top_voteup_comment = max(selecting, key=lambda x: x.vote_count)
#     conversation = list(top_voteup_comment.conversation)
    conversation = [top_voteup_comment]
    
    yield [_parse_comment(comment) for comment in conversation]
    group_limit -= 1
    import time
    time.sleep(1)
    for c in conversation:
      selected_ids.add(c.id)



In [137]:
v = valuable_conversations(answer)
list(v)

RetryError: HTTPSConnectionPool(host='api.zhihu.com', port=443): Max retries exceeded with url: /comments/143137301/conversation (Caused by ResponseError('too many 400 error responses',))

In [7]:

"""返回 aid 用于拼接 url get 该回答的评论
<div tabindex="-1" class="zm-item-answer" itemscope="" itemtype="http://schema.org/Answer" data-aid="14852408" data-atoken="48635152" data-collapsed="0" data-created="1432285826" data-deleted="0" data-helpful="1" data-isowner="0" data-score="34.1227812032">
"""
headers = {
    'User-agent': 'Mozilla/5.0',
}


url = 'http://www.zhihu.com/question/38860589/answer/79205923'
from pyquery import PyQuery
import requests
r = requests.get(url, headers=headers)
print(r.text)


<!DOCTYPE html>
<html lang="zh-CN" dropEffect="none" class="no-js no-auth ">
<head>
<meta charset="utf-8" />

<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
<meta name="renderer" content="webkit" />
<meta http-equiv="X-ZA-Response-Id" content="8704e65689b6402596a613183c400faf">

<title>如果太阳系是一个双恒星的星系，那地球应该是什么样的运转轨道，地球人的生活会是什么样的？ - Mandelbrot 的回答 - 知乎</title>

<meta name="apple-itunes-app" content="app-id=432274380, app-argument=zhihu://answers/79205923">

<meta property="og:description" content="如果太阳和另一颗恒星构成一个双星系统，地球和上面的生物以及人类文明肯定会和现在完全不同。在这篇回答中，我准备详细讲述我想象中的双星地球上，人类文明的命运。不过，在详细分析这个问题以前，我们有必要先了解一下双星。（如果你只想看科幻，可以直接…" />



<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1"/>
<meta http-equiv="mobile-agent" content="format=html5;url=http://www.zhihu.com/question/38860589/answer/79205923">
<meta id="znonce" name="znonce" content="c68532b3c5d847aeb52a80d813b8c993">



<link rel="apple-touch-icon" href="http://static.zhihu.com/static/revved

In [8]:
# div = self.soup.find('div', class_='zm-item-answer')
# return div['data-aid']
doc = PyQuery(r.text).find('div.zm-item-answer')
print(doc)

<div tabindex="-1" class="zm-item-answer  zm-item-expanded" itemscope="" itemtype="http://schema.org/Answer" data-aid="27109662" data-atoken="79205923" data-collapsed="0" data-created="1451442368" data-deleted="0" data-helpful="1" data-isowner="0" data-copyable="1">
<link itemprop="url" href="/question/38860589/answer/79205923"/>
<meta itemprop="answer-id" content="27109662"/>
<meta itemprop="answer-url-token" content="79205923"/>
<a class="zg-anchor-hidden" name="answer-27109662"/>


<div class="zm-votebar">
<button class="up ">
<i class="icon vote-arrow"/>
<span class="count">2897</span>
<span class="label sr-only">赞同</span>
</button>
<button class="down ">
<i class="icon vote-arrow"/>
<span class="label sr-only">反对</span>
</button>
</div>


<div class="answer-head">
<div class="zm-item-answer-author-info">

<a class="zm-item-link-avatar avatar-link" href="/people/mandelbrot-11" target="_blank" data-tip="p$t$mandelbrot-11"><img src="https://pic3.zhimg.com/0f9d7b8fa30ea14da94a9f0f4f80

In [12]:
doc.attr('data-aid')


'27109662'

In [13]:
def comment_list_id(url):
  """返回 aid 用于拼接 url get 该回答的评论
  <div tabindex="-1" class="zm-item-answer" itemscope="" itemtype="http://schema.org/Answer" data-aid="14852408" data-atoken="48635152" data-collapsed="0" data-created="1432285826" data-deleted="0" data-helpful="1" data-isowner="0" data-score="34.1227812032">
  """
  from pyquery import PyQuery
  import requests
  headers = {
    'User-agent': 'Mozilla/5.0',
  }
  r = requests.get(url, headers=headers)
  return PyQuery(r.text).find('div.zm-item-answer').attr('data-aid')

url = 'https://www.zhihu.com/question/29400357/answer/82408466'
aid = comment_list_id(url)
aid

'28392999'

In [15]:
comment_box_link = 'http://www.zhihu.com/node/AnswerCommentBoxV2?params=%7B%22answer_id%22%3A%22{}%22%2C%22load_all%22%3Atrue%7D'.format(aid)
headers = {
'User-agent': 'Mozilla/5.0',
}
r = requests.get(comment_box_link, headers=headers)
r.text

'<div class="zm-comment-box" data-count="4">\n<i class="icon icon-spike zm-comment-bubble"></i>\n<a class="zg-anchor-hidden" name="comment-0"></a>\n<div class="zm-comment-list">\n<div class="zm-item-comment" data-id="114689834" tabindex="-1" aria-label="评论">\n<a class="zg-anchor-hidden" name="comment-114689834"></a>\n<a title="张雨萌"\ndata-tip="p$t$zhang-yu-meng-7"\nclass="zm-item-link-avatar"\nhref="/people/zhang-yu-meng-7">\n<img src="https://pic2.zhimg.com/f67548ff9d894b4813f414f5e22a3229_s.jpg" class="zm-item-img-avatar">\n</a>\n<div class="zm-comment-content-wrap">\n<div class="zm-comment-hd">\n<a data-tip="p$t$zhang-yu-meng-7" href="http://www.zhihu.com/people/zhang-yu-meng-7" class="zg-link" title="张雨萌"\n>张雨萌</a>\n\n</div>\n<div class="zm-comment-content">\n搬板凳等下文\n</div>\n<div class="zm-comment-ft">\n<span class="date">2016-01-20</span>\n\n\n<a href="#" class="reply zm-comment-op-link" name="reply_comment">\n<i class="zg-icon zg-icon-comment-reply"></i>回复</a>\n\n\n<a href="#" cla

In [20]:
doc = PyQuery(r.text)
for div in doc.find('div.zm-item-comment'):
    div = PyQuery(div)
    cid = div.attr('data-id')
#     likes = int(div.find('span', class_='like-num').find('em').text))

<Element div at 0x3cf78a0>
114689834
<Element div at 0x3cf78d0>
114728412
<Element div at 0x3cf7900>
114771158
<Element div at 0x3cf7930>
137769633


In [28]:
div = list(doc.find('div.zm-item-comment'))[0]
div = PyQuery(div)
div.attr('data-id')
int(div.find('span.like-num').find('em').text())
div.find('div.zm-comment-content').text()
div.find('div.zm-comment-hd').text().replace('\n', ' ')

'张雨萌'

In [30]:
from zhihu_oauth import ZhihuClient
TOKEN_FILE = 'token.pkl'
client = ZhihuClient()
client.load_token(TOKEN_FILE)
from pprint import pprint
url='http://www.zhihu.com/node/AnswerCommentBoxV2?params=%7B%22answer_id%22%3A%2227109662%22%2C%22load_all%22%3Atrue%7D'

r = client.test_api('GET', url)
r.content

b'<div class="zm-comment-box" data-count="243">\n<i class="icon icon-spike zm-comment-bubble"></i>\n<a class="zg-anchor-hidden" name="comment-0"></a>\n<div class="zm-comment-list">\n<div class="zm-item-comment" data-id="110995851" tabindex="-1" aria-label="\xe8\xaf\x84\xe8\xae\xba">\n<a class="zg-anchor-hidden" name="comment-110995851"></a>\n<a title="Phoenix"\ndata-tip="p$t$phoenix-37"\nclass="zm-item-link-avatar"\nhref="/people/phoenix-37">\n<img src="https://pic4.zhimg.com/d42a1ad87_s.jpg" class="zm-item-img-avatar">\n</a>\n<div class="zm-comment-content-wrap">\n<div class="zm-comment-hd">\n<a data-tip="p$t$phoenix-37" href="http://www.zhihu.com/people/phoenix-37" class="zg-link" title="Phoenix"\n>Phoenix</a>\n\n</div>\n<div class="zm-comment-content">\n\xe5\xa4\xaa\xe9\x95\xbf\xe4\xba\x86\xef\xbc\x8c\xe7\x9c\x8b\xe5\xae\x8c\xe9\x85\x8d\xe5\x9b\xbe\xe4\xb9\x8b\xe5\x90\x8e\xe7\x82\xb9\xe4\xba\x86\xe6\x84\x9f\xe8\xb0\xa2\xef\xbc\x8c\xe5\x9b\x9e\xe5\xa4\xb4\xe6\x85\xa2\xe6\x85\xa2\xe7\